In [ ]:
pip install CatBoost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 120.0 MB/s 


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 27.4 MB/s 
     |████████████████████████████████| 209 kB 92.9 MB/s 
     |████████████████████████████████| 81 kB 12.1 MB/s 
     |████████████████████████████████| 78 kB 9.7 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 147 kB 101.9 MB/s 
     |████████████████████████████████| 112 kB 85.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=480a271bff15fb429b06ec6ba0ea2145d7d0b5944d76eba0845c6430172c283e
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


from catboost import CatBoostClassifier, Pool

import pandas as pd, optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

#### 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/오일공모전/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/오일공모전/test.csv')

In [ ]:
train_data

,Unnamed: 0,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,0,TRAIN_00000,COMPONENT3,1486.0,2011.0,7.0,200.0,0.0,3.0,93.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.9,154.0,75.0,0
1,1,TRAIN_00001,COMPONENT2,1350.0,2021.0,51.0,375.0,0.0,2.0,19.0,...,2.0,4.0,6.0,216.0,1454.0,0.0,12.9,44.0,652.0,0
2,2,TRAIN_00002,COMPONENT2,2415.0,2015.0,2.0,200.0,0.0,110.0,1.0,...,0.0,3.0,39.0,11261.0,41081.0,0.0,13.3,72.6,412.0,1
3,3,TRAIN_00003,COMPONENT3,7389.0,2010.0,2.0,200.0,0.0,8.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,133.3,7.0,0
4,4,TRAIN_00004,COMPONENT3,3954.0,2015.0,4.0,200.0,0.0,1.0,157.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.1,133.1,128.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,14090,TRAIN_14090,COMPONENT3,1616.0,2014.0,8.0,200.0,0.0,2.0,201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,135.4,16.0,0
14091,14091,TRAIN_14091,COMPONENT1,2784.0,2013.0,2.0,200.0,0.0,3.0,85.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.5,117.5,1408.0,0
14092,14092,TRAIN_14092,COMPONENT3,1788.0,2008.0,9.0,550.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.6,54.0,1301.0,0
14093,14093,TRAIN_14093,COMPONENT2,2498.0,2009.0,19.0,550.0,0.0,2.0,4.0,...,7.0,8.0,100.0,1625.0,18890.0,0.0,13.2,44.3,652.0,0


In [ ]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

###  Data Preprocessing
#### 1. Train / Validation 분할
#### 2. log1p
#### 3. Data label-encoding, scaling

In [ ]:
# train x, y 분리
x_train_data = train_data.loc[:,['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
y_train_data = pd.DataFrame(train_data['Y_LABEL'])

In [ ]:
#test /// 데이터 나누기
test_log = test_data.loc[:,['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
test_lab = test_data.loc[:,['COMPONENT_ARBITRARY','YEAR']]
test_log

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2192,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,2794,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,1982,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,1404,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,8225,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,1714,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,4131,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,4325,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,1364,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [ ]:
x_train_data

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,COMPONENT3,1486.0,2011.0,200.0,0.0,0.0,13.0,78.0,888.0,0.0,16.0,1.0,6.0,8504.0,5.0,0.0,154.0,75.0
1,COMPONENT2,1350.0,2021.0,375.0,0.0,0.0,0.0,31.0,2.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,44.0,652.0
2,COMPONENT2,2415.0,2015.0,200.0,0.0,0.0,1.0,2.0,4.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,72.6,412.0
3,COMPONENT3,7389.0,2010.0,200.0,0.0,0.0,0.0,1.0,37.0,0.0,1.0,0.0,0.0,44.0,0.0,0.0,133.3,7.0
4,COMPONENT3,3954.0,2015.0,200.0,0.0,0.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,217.0,0.0,0.0,133.1,128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616.0,2014.0,200.0,0.0,0.0,0.0,3.0,23.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,135.4,16.0
14091,COMPONENT1,2784.0,2013.0,200.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,224.0,0.0,9.0,0.0,0.0,117.5,1408.0
14092,COMPONENT3,1788.0,2008.0,550.0,0.0,0.0,4.0,7.0,415.0,0.0,7.0,10.0,1.0,645.0,0.0,0.0,54.0,1301.0
14093,COMPONENT2,2498.0,2009.0,550.0,0.0,0.0,0.0,170.0,19.0,0.0,0.0,1.0,0.0,11.0,0.0,0.0,44.3,652.0


In [ ]:
#train /// 데이터 나누기
x_log = x_train_data.loc[:,['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
x_lab = x_train_data.loc[:,['COMPONENT_ARBITRARY','YEAR']]
x_log

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412
3,7389,200,0,0,0,1,37,0.0,1,0,0,44,0,0,133.3,7
4,3954,200,0,0,0,0,71,0.0,0,0,0,217,0,0,133.1,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,1616,200,0,0,0,3,23,0.0,0,0,0,35,0,0,135.4,16
14091,2784,200,0,0,0,2,12,0.0,0,224,0,9,0,0,117.5,1408
14092,1788,550,0,0,4,7,415,0.0,7,10,1,645,0,0,54.0,1301
14093,2498,550,0,0,0,170,19,0.0,0,1,0,11,0,0,44.3,652


#### 2. log1p

In [ ]:
# x_train //// COMPONENT_ARBITRARY, YEAR, 제외 log1p
train_x1 = np.log1p(x_log)
train_x1

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,7.304516,5.303305,0.0,0.0,2.639057,4.369448,6.790097,0.0,2.833213,0.693147,1.945910,9.048410,1.791759,0.0,5.043425,4.330733
1,7.208600,5.929589,0.0,0.0,0.000000,3.465736,1.098612,0.0,0.000000,0.000000,0.000000,2.995732,0.000000,0.0,3.806662,6.481577
2,7.789869,5.303305,0.0,0.0,0.693147,1.098612,1.609438,0.0,0.000000,0.000000,0.000000,2.890372,0.000000,0.0,4.298645,6.023448
3,8.907883,5.303305,0.0,0.0,0.000000,0.693147,3.637586,0.0,0.693147,0.000000,0.000000,3.806662,0.000000,0.0,4.900076,2.079442
4,8.282736,5.303305,0.0,0.0,0.000000,0.000000,4.276666,0.0,0.000000,0.000000,0.000000,5.384495,0.000000,0.0,4.898586,4.859812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,7.388328,5.303305,0.0,0.0,0.000000,1.386294,3.178054,0.0,0.000000,0.000000,0.000000,3.583519,0.000000,0.0,4.915592,2.833213
14091,7.932003,5.303305,0.0,0.0,0.000000,1.098612,2.564949,0.0,0.000000,5.416100,0.000000,2.302585,0.000000,0.0,4.774913,7.250636
14092,7.489412,6.311735,0.0,0.0,1.609438,2.079442,6.030685,0.0,2.079442,2.397895,0.693147,6.470800,0.000000,0.0,4.007333,7.171657
14093,7.823646,6.311735,0.0,0.0,0.000000,5.141664,2.995732,0.0,0.000000,0.693147,0.000000,2.484907,0.000000,0.0,3.813307,6.481577


In [ ]:
# test //// COMPONENT_ARBITRARY, YEAR, 제외 log1p
test_1 = np.log1p(test_log)
test_1

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,7.693026,5.303305,0.000000,0.0,0.000000,0.693147,2.564949,0.0,0.000000,0.000000,0.000000,2.397895,0.000000,0.0,4.525044,6.995766
1,7.935587,5.303305,0.000000,0.0,1.098612,0.693147,5.631212,0.0,1.386294,0.000000,0.000000,7.913155,0.693147,0.0,4.851249,2.564949
2,7.592366,5.303305,0.000000,0.0,0.000000,2.833213,1.791759,0.0,0.000000,0.000000,0.000000,2.484907,0.000000,0.0,3.813307,6.572283
3,7.247793,5.303305,0.000000,0.0,1.386294,1.609438,5.099866,0.0,1.609438,1.386294,0.000000,8.988196,0.000000,0.0,4.968423,4.553877
4,9.015055,5.303305,0.000000,0.0,0.000000,1.945910,2.639057,0.0,0.000000,0.000000,0.000000,2.833213,0.000000,0.0,4.165114,6.152733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,7.447168,5.303305,0.000000,0.0,1.386294,4.875197,6.954639,0.0,4.189655,1.791759,1.098612,7.600902,0.000000,0.0,4.279440,7.059618
6037,8.326517,5.303305,0.000000,0.0,1.791759,1.098612,6.602588,0.0,1.791759,0.000000,0.000000,6.542472,0.000000,0.0,4.933034,2.639057
6038,8.372399,5.303305,0.000000,0.0,0.000000,0.000000,3.988984,0.0,0.000000,0.000000,0.000000,4.248495,0.000000,0.0,5.714393,2.639057
6039,7.218910,5.303305,0.000000,0.0,0.000000,4.143135,1.098612,0.0,0.000000,0.000000,0.000000,1.791759,0.000000,0.0,3.953165,6.171701


#### 3. Data label-encoding, scaling

In [ ]:
# train//// StandardScaler
standardScaler = StandardScaler()
scaled_train_x = standardScaler.fit_transform(train_x1)
print(scaled_train_x)

[[-0.91275182 -0.62392797 -0.15374557 ... -0.15450672  1.00786117
  -0.61423005]
 [-1.07682894  0.40501755 -0.15374557 ... -0.15450672 -1.93938741
   0.60686173]
 [-0.08248804 -0.62392797 -0.15374557 ... -0.15450672 -0.76697582
   0.34676929]
 ...
 [-0.59646102  1.0328588  -0.15374557 ... -0.15450672 -1.46118212
   0.99863851]
 [-0.0247071   1.0328588  -0.15374557 ... -0.15450672 -1.92355323
   0.60686173]
 [-0.49078658 -0.62392797 -0.15374557 ... -0.15450672 -1.78558985
   0.57097447]]


In [ ]:
#train /// 스케일링데이터 데이터프레임으로 만들어주기
scaled_train_x = pd.DataFrame(scaled_train_x)

In [ ]:
#train /// 스케일링한 데이터 컬럼명 변경
scaled_train_x.columns = ['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']
scaled_train_x

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,-0.912752,-0.623928,-0.153746,-0.137993,2.141433,1.477746,1.851314,-0.09694,2.364083,-0.372968,3.059688,2.758639,3.210181,-0.154507,1.007861,-0.614230
1,-1.076829,0.405018,-0.153746,-0.137993,-0.892546,0.909950,-1.761179,-0.09694,-0.786784,-0.776658,-0.444582,-0.524164,-0.264003,-0.154507,-1.939387,0.606862
2,-0.082488,-0.623928,-0.153746,-0.137993,-0.095673,-0.577297,-1.436948,-0.09694,-0.786784,-0.776658,-0.444582,-0.581308,-0.264003,-0.154507,-0.766976,0.346769
3,1.830033,-0.623928,-0.153746,-0.137993,-0.892546,-0.832048,-0.149644,-0.09694,-0.015923,-0.776658,-0.444582,-0.084338,-0.264003,-0.154507,0.666255,-1.892349
4,0.760631,-0.623928,-0.153746,-0.137993,-0.892546,-1.267548,0.255992,-0.09694,-0.786784,-0.776658,-0.444582,0.771434,-0.264003,-0.154507,0.662704,-0.313858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.769380,-0.623928,-0.153746,-0.137993,-0.892546,-0.396549,-0.441318,-0.09694,-0.786784,-0.776658,-0.444582,-0.205365,-0.264003,-0.154507,0.703230,-1.464412
14091,0.160653,-0.623928,-0.153746,-0.137993,-0.892546,-0.577297,-0.830467,-0.09694,-0.786784,2.377687,-0.444582,-0.900107,-0.264003,-0.154507,0.367987,1.043477
14092,-0.596461,1.032859,-0.153746,-0.137993,0.957736,0.038951,1.369301,-0.09694,1.525800,0.619880,0.803664,1.360615,-0.264003,-0.154507,-1.461182,0.998639
14093,-0.024707,1.032859,-0.153746,-0.137993,-0.892546,1.962923,-0.557041,-0.09694,-0.786784,-0.372968,-0.444582,-0.801221,-0.264003,-0.154507,-1.923553,0.606862


In [ ]:
#train test //// 데이터 합치기
train_x = pd.concat([x_lab, scaled_train_x], axis = 1)
test1 = pd.concat([test_lab, test_1], axis = 1)

In [ ]:
train_x

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,COMPONENT3,2011,-0.912752,-0.623928,-0.153746,-0.137993,2.141433,1.477746,1.851314,-0.09694,2.364083,-0.372968,3.059688,2.758639,3.210181,-0.154507,1.007861,-0.614230
1,COMPONENT2,2021,-1.076829,0.405018,-0.153746,-0.137993,-0.892546,0.909950,-1.761179,-0.09694,-0.786784,-0.776658,-0.444582,-0.524164,-0.264003,-0.154507,-1.939387,0.606862
2,COMPONENT2,2015,-0.082488,-0.623928,-0.153746,-0.137993,-0.095673,-0.577297,-1.436948,-0.09694,-0.786784,-0.776658,-0.444582,-0.581308,-0.264003,-0.154507,-0.766976,0.346769
3,COMPONENT3,2010,1.830033,-0.623928,-0.153746,-0.137993,-0.892546,-0.832048,-0.149644,-0.09694,-0.015923,-0.776658,-0.444582,-0.084338,-0.264003,-0.154507,0.666255,-1.892349
4,COMPONENT3,2015,0.760631,-0.623928,-0.153746,-0.137993,-0.892546,-1.267548,0.255992,-0.09694,-0.786784,-0.776658,-0.444582,0.771434,-0.264003,-0.154507,0.662704,-0.313858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,2014,-0.769380,-0.623928,-0.153746,-0.137993,-0.892546,-0.396549,-0.441318,-0.09694,-0.786784,-0.776658,-0.444582,-0.205365,-0.264003,-0.154507,0.703230,-1.464412
14091,COMPONENT1,2013,0.160653,-0.623928,-0.153746,-0.137993,-0.892546,-0.577297,-0.830467,-0.09694,-0.786784,2.377687,-0.444582,-0.900107,-0.264003,-0.154507,0.367987,1.043477
14092,COMPONENT3,2008,-0.596461,1.032859,-0.153746,-0.137993,0.957736,0.038951,1.369301,-0.09694,1.525800,0.619880,0.803664,1.360615,-0.264003,-0.154507,-1.461182,0.998639
14093,COMPONENT2,2009,-0.024707,1.032859,-0.153746,-0.137993,-0.892546,1.962923,-0.557041,-0.09694,-0.786784,-0.372968,-0.444582,-0.801221,-0.264003,-0.154507,-1.923553,0.606862


In [ ]:
test1

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,COMPONENT1,2016,7.693026,5.303305,0.000000,0.0,0.000000,0.693147,2.564949,0.0,0.000000,0.000000,0.000000,2.397895,0.000000,0.0,4.525044,6.995766
1,COMPONENT3,2011,7.935587,5.303305,0.000000,0.0,1.098612,0.693147,5.631212,0.0,1.386294,0.000000,0.000000,7.913155,0.693147,0.0,4.851249,2.564949
2,COMPONENT2,2010,7.592366,5.303305,0.000000,0.0,0.000000,2.833213,1.791759,0.0,0.000000,0.000000,0.000000,2.484907,0.000000,0.0,3.813307,6.572283
3,COMPONENT3,2009,7.247793,5.303305,0.000000,0.0,1.386294,1.609438,5.099866,0.0,1.609438,1.386294,0.000000,8.988196,0.000000,0.0,4.968423,4.553877
4,COMPONENT2,2013,9.015055,5.303305,0.000000,0.0,0.000000,1.945910,2.639057,0.0,0.000000,0.000000,0.000000,2.833213,0.000000,0.0,4.165114,6.152733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,COMPONENT3,2014,7.447168,5.303305,0.000000,0.0,1.386294,4.875197,6.954639,0.0,4.189655,1.791759,1.098612,7.600902,0.000000,0.0,4.279440,7.059618
6037,COMPONENT3,2016,8.326517,5.303305,0.000000,0.0,1.791759,1.098612,6.602588,0.0,1.791759,0.000000,0.000000,6.542472,0.000000,0.0,4.933034,2.639057
6038,COMPONENT3,2014,8.372399,5.303305,0.000000,0.0,0.000000,0.000000,3.988984,0.0,0.000000,0.000000,0.000000,4.248495,0.000000,0.0,5.714393,2.639057
6039,COMPONENT2,2013,7.218910,5.303305,0.000000,0.0,0.000000,4.143135,1.098612,0.0,0.000000,0.000000,0.000000,1.791759,0.000000,0.0,3.953165,6.171701


In [ ]:
# train, test /// COMPONENT_ARBITRARY, YEAR 라벨인코딩
le =LabelEncoder()
x_train_data_le = train_x
test = test1

In [ ]:
# train test /// COMPONENT_ARBITRARY 라벨인코딩 후 합치기
x_train_data_le["COMPONENT_ARBITRARY"] = le.fit_transform(train_x["COMPONENT_ARBITRARY"])
x_train_data_le["YEAR"] = le.fit_transform(train_x["YEAR"])

test["COMPONENT_ARBITRARY"] = le.fit_transform(test1["COMPONENT_ARBITRARY"])
test["YEAR"] = le.fit_transform(test1["YEAR"])

#### 데이터 정리 및 확인

In [ ]:
# train, test 컬럼순서 맞추기
X_train = x_train_data_le[['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
test = test[['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]

In [ ]:
X_train

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2,-0.912752,4,-0.623928,-0.153746,-0.137993,2.141433,1.477746,1.851314,-0.09694,2.364083,-0.372968,3.059688,2.758639,3.210181,-0.154507,1.007861,-0.614230
1,1,-1.076829,14,0.405018,-0.153746,-0.137993,-0.892546,0.909950,-1.761179,-0.09694,-0.786784,-0.776658,-0.444582,-0.524164,-0.264003,-0.154507,-1.939387,0.606862
2,1,-0.082488,8,-0.623928,-0.153746,-0.137993,-0.095673,-0.577297,-1.436948,-0.09694,-0.786784,-0.776658,-0.444582,-0.581308,-0.264003,-0.154507,-0.766976,0.346769
3,2,1.830033,3,-0.623928,-0.153746,-0.137993,-0.892546,-0.832048,-0.149644,-0.09694,-0.015923,-0.776658,-0.444582,-0.084338,-0.264003,-0.154507,0.666255,-1.892349
4,2,0.760631,8,-0.623928,-0.153746,-0.137993,-0.892546,-1.267548,0.255992,-0.09694,-0.786784,-0.776658,-0.444582,0.771434,-0.264003,-0.154507,0.662704,-0.313858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,2,-0.769380,7,-0.623928,-0.153746,-0.137993,-0.892546,-0.396549,-0.441318,-0.09694,-0.786784,-0.776658,-0.444582,-0.205365,-0.264003,-0.154507,0.703230,-1.464412
14091,0,0.160653,6,-0.623928,-0.153746,-0.137993,-0.892546,-0.577297,-0.830467,-0.09694,-0.786784,2.377687,-0.444582,-0.900107,-0.264003,-0.154507,0.367987,1.043477
14092,2,-0.596461,1,1.032859,-0.153746,-0.137993,0.957736,0.038951,1.369301,-0.09694,1.525800,0.619880,0.803664,1.360615,-0.264003,-0.154507,-1.461182,0.998639
14093,1,-0.024707,2,1.032859,-0.153746,-0.137993,-0.892546,1.962923,-0.557041,-0.09694,-0.786784,-0.372968,-0.444582,-0.801221,-0.264003,-0.154507,-1.923553,0.606862


In [ ]:
y_train_data

,Y_LABEL
0,0
1,0
2,1
3,0
4,0
...,...
14090,0
14091,0
14092,0
14093,0


In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,0,7.693026,9,5.303305,0.000000,0.0,0.000000,0.693147,2.564949,0.0,0.000000,0.000000,0.000000,2.397895,0.000000,0.0,4.525044,6.995766
1,2,7.935587,4,5.303305,0.000000,0.0,1.098612,0.693147,5.631212,0.0,1.386294,0.000000,0.000000,7.913155,0.693147,0.0,4.851249,2.564949
2,1,7.592366,3,5.303305,0.000000,0.0,0.000000,2.833213,1.791759,0.0,0.000000,0.000000,0.000000,2.484907,0.000000,0.0,3.813307,6.572283
3,2,7.247793,2,5.303305,0.000000,0.0,1.386294,1.609438,5.099866,0.0,1.609438,1.386294,0.000000,8.988196,0.000000,0.0,4.968423,4.553877
4,1,9.015055,6,5.303305,0.000000,0.0,0.000000,1.945910,2.639057,0.0,0.000000,0.000000,0.000000,2.833213,0.000000,0.0,4.165114,6.152733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,2,7.447168,7,5.303305,0.000000,0.0,1.386294,4.875197,6.954639,0.0,4.189655,1.791759,1.098612,7.600902,0.000000,0.0,4.279440,7.059618
6037,2,8.326517,9,5.303305,0.000000,0.0,1.791759,1.098612,6.602588,0.0,1.791759,0.000000,0.000000,6.542472,0.000000,0.0,4.933034,2.639057
6038,2,8.372399,7,5.303305,0.000000,0.0,0.000000,0.000000,3.988984,0.0,0.000000,0.000000,0.000000,4.248495,0.000000,0.0,5.714393,2.639057
6039,1,7.218910,6,5.303305,0.000000,0.0,0.000000,4.143135,1.098612,0.0,0.000000,0.000000,0.000000,1.791759,0.000000,0.0,3.953165,6.171701


#### CatBoost

In [ ]:
# catboost

# k-fold
kfold = StratifiedKFold(n_splits=3,random_state=123, shuffle=True)

def cat_objective(trial):
    params = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        "learning_rate" : trial.suggest_float("learning_rate", 0.0001, 0.003, log=True),
        "reg_lambda" : trial.suggest_float("reg_alpha", 0.01, 1, log=True),
        'random_strength': trial.suggest_float('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_float('bagging_temperature', 0.1, 10, log=True),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0)
    }
    cat = CatBoostClassifier(**params) 
    score= cross_val_score(cat, X_train, y_train_data, cv=kfold, scoring="f1", n_jobs=-1)
    accuracy = score.mean()
    return accuracy
    
cat_study = optuna.create_study(direction="maximize")
cat_study.optimize(cat_objective, n_trials=30) 
print(cat_study.best_trial.params)
print(cat_study.best_trial.values)
# optimization
optuna.visualization.plot_param_importances(cat_study)
opt_cat = CatBoostClassifier( 
                        iterations = cat_study.best_param["iterations"],
                        od_wait = cat_study.best_param["od_wait"],
                        learning_rate = cat_study.best_param["learning_rate"],
                        reg_lambda = cat_study.best_param["reg_lambda"],
                        random_strength = cat_study.best_param["random_strength"],
                        depth = cat_study.best_param["depth"],
                        min_data_in_leaf = cat_study.best_param["min_data_in_leaf"],
                        leaf_estimation_iterations = cat_study.best_param["leaf_estimation_iterations"],
                        bagging_temperature = cat_study.best_param["bagging_temperature"],
                        colsample_bylevel = cat_study.best_param["colsample_bylevel"])

opt_cat.fit(X_train, y_train_data)
print(opt_cat.predict(test))
print(f1_score(y_train_data, opt_cat.predict(test)))

[I 2022-12-10 04:44:26,026] A new study created in memory with name: no-name-f972afe1-4e08-4e57-81cb-5c18dd88e677
[I 2022-12-10 04:45:01,034] Trial 0 finished with value: 0.0 and parameters: {'iterations': 10510, 'od_wait': 1266, 'learning_rate': 0.0007501894608150215, 'reg_alpha': 0.02453039687964241, 'random_strength': 26.840703485682393, 'depth': 3, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 2, 'bagging_temperature': 1.6676720809733485, 'colsample_bylevel': 0.4837500841391474}. Best is trial 0 with value: 0.0.
[I 2022-12-10 04:54:25,900] Trial 1 finished with value: 0.001658374792703151 and parameters: {'iterations': 7260, 'od_wait': 1249, 'learning_rate': 0.00040005408779388116, 'reg_alpha': 0.6569619547695599, 'random_strength': 47.96395503656483, 'depth': 13, 'min_data_in_leaf': 27, 'leaf_estimation_iterations': 3, 'bagging_temperature': 3.675913491350506, 'colsample_bylevel': 0.7757501059868053}. Best is trial 1 with value: 0.001658374792703151.
[I 2022-12-10 04:58:04

KeyboardInterrupt: ignored

#### pickle

In [ ]:
joblib.dump(opt_cat, 'saved_cat.pkl')